What’s wrong?

Always returns zenith… list of what I think may be the issue. 

1. Could be due to indexing, but I did an equation for this, along with working on it today

2. Could be because I do not include the same conditionals I have in my slower version. 

3. The array of chi squared vals breaks somewhere, automatically returning zeros. Not sure why though.


I could resolve 2 by figuring out how to replace values in an array, need to be able to go into specific indices, and I’m not sure if there’s a way to do that without iterating, rendering this entire thing worthless. 



Thanks!


To run through this stuff, just hit run on the cells in order. 

The first cell contains the class I use to generate GRBs

In [1]:

#Needs to know how to point to healpy 
import healpy as hp

class Sky():
    """
    Generates an array of GRB's given 
    certains strength at different sky positions.
    
    Output should be an array. 
    """
    def __init__(self,NSIDE,strength):

        #depending on NSIDE, there will be anywhere from 12 to infinite spots on the sky w/ GRBs
        self.Ao = strength
        self.pixels = hp.nside2npix(NSIDE)

        #want to convert these pixels into theta phi coords. 
        self.sourceangs = []
        for i in range(self.pixels):
            self.sourceangs.append(hp.pix2ang(NSIDE,i))

    def say_Ao(self):
        """
        """

        print("The GRBs being tested will be " + str(self.Ao) + " counts strong.")
        

The next cell contains a class I call "SlowCube," these are my original simulations, depending on the iterative process, sacrificing speed for actually working. Note that they use the original function "solver," which is the function I use to calculate the minimum chi square and the corresponding sky position.

In [2]:
import numpy as np
import healpy as hp
import burstutils as bf
import random as rand
import statistics as s

class SlowCube():

    def __init__(self,background,dettilt,alternating=False):
        if alternating == False:
            self.tilt = np.deg2rad(dettilt)
            self.tiltA = self.tiltB = self.tiltC = self.tiltD = self.tilt
        
        else:
            self.tiltB = np.deg2rad(float(input("Please enter the second tilt (deg) ")))
        
            self.tiltC = self.tiltA = dettilt
            self.tiltD = self.tiltB
        self.zenith = [0 , 0]
        self.bg = background

            
    
    #make the normal vectors!
        #self.detA = [ zenith[0] + self.tiltA , zenith[1] ] 
        #self.detB = [ zenith[0] + self.tiltB , zenith[1] + np.pi/2 ] 
       # self.detC = [ zenith[0] + self.tiltC , zenith[1] + np.pi ] 
        #self.detD = [ zenith[0] + self.tiltD , zenith[1] + 3*np.pi/2 ] 

        #self.Anorm = hp.ang2vec(self.detA[0],self.detA[1])
        #self.Bnorm = hp.ang2vec(self.detB[0],self.detB[1])
        #self.Cnorm = hp.ang2vec(self.detC[0],self.detC[1])
        #self.Dnorm = hp.ang2vec(self.detD[0],self.detD[1])

    
       # self.dets = [self.Anorm,self.Bnorm,self.Cnorm,self.Dnorm] 
        
    @property
    def detA(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltA , self.zenith[1] ]
    @property 
    def detB(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltB , self.zenith[1] + np.pi/2 ]
    @property
    def detC(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltC , self.zenith[1] + np.pi ]
    @property 
    def detD(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltD , self.zenith[1] + 3*np.pi/2 ]
    @property
    def normA(self):
        return  hp.ang2vec(self.detA[0],self.detA[1])
    @property 
    def normB(self):
        return  hp.ang2vec(self.detB[0],self.detB[1])
    @property
    def normC(self):
        return  hp.ang2vec(self.detC[0],self.detC[1])
    @property 
    def normD(self):
        return  hp.ang2vec(self.detD[0],self.detD[1])

    
    @property
    def dets(self):
        return [self.normA,self.normB,self.normC,self.normD] 
    
    
    
    def response2GRB(self, GRB, samples = 1, test = False):   #is this how I inherit? 
        #first need to include the GRB.
       
        """
        Using least squares regression, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
        
        An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 

        Returns
        ----------
        localizationerrors : array

        numpy array that contains the average localization uncertainty at each sky position. 

        
        """
        
        if test:
            sample = 1
        else:
            sample = len(GRB.sourceangs)
        
        self.localizationerrors = []    
        for i in range(sample):
            sourceAng = GRB.sourceangs[i]
            print("Testing " + str(np.rad2deg(sourceAng)))
           #this check passes.       

            
           # print("Testing at " + str(np.rad2deg(GRB.sourceangs)))
            sourcexyz = hp.ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst
            loop = 0 #I'm going to want to sample each sky position more than once,
                    #here's where I define how many times that is
            locunc = []
            while loop<samples:  #how often a sky position is sampled, only 1 for now, the larger it is though, the slower the sim. 
                detcounts = []  #number of counts incident in each detector. 
                for j in range(len(self.dets)):
                    sep=bf.angle(sourcexyz,self.dets[j])
                   # print("separation is " + str(np.rad2deg(sep)))
                   #this check passes.  
               
                    if sep < np.pi/2: # meaning if >90, would not be facing detector.
                        dtheory=bf.response(GRB.Ao,bf.angle(sourcexyz,self.dets[j]),self.bg) #still need to define strength, brb and gonna do that 
                    else: #like I was saying, has to face it!
                        dtheory = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                    counts = dtheory #another artifact, incl this background effect somewhere
                    unccounts = np.sqrt(counts)
                    detactual = rand.gauss(counts,unccounts)  #there is a lot of noise, present, updating it now. 
                    if detactual-self.bg < 0:
                        detactual = 0
                    
                    detcounts.append(counts)
                
                
                self.detcounts = detcounts
                
             #   coarsethetaloc,coarsephiloc,coarseAo, coarsechis = bf.solver(detcounts,self.dets,0,90,0,360,2,2,self.bg)
                finethetaloc,finephiloc,fineAo, finechis = bf.solver(detcounts,self.dets,0,90,0,360,2,2,self.bg)
                
               # print(coarsechis)
                print(finechis)
                
                if finethetaloc > 180:
                #    print("it recovered an unrealistic answer, skip")
                    break
                elif finethetaloc < 0:
                    #print("Same issue, there are limits to theta that are broken here. ")
                    break 
                recpos = [finethetaloc,finephiloc]
              #  print("Recovered position at " + str(recpos))
                
                recvec = hp.ang2vec(np.deg2rad(finethetaloc),np.deg2rad(finephiloc))
                locunc.append(bf.angle(sourcexyz,recvec))
              #   print("loc unc: " + str(np.rad2deg(bf.angle(sourcexyz,recvec))) + " deg")
                loop+=1
          #  print("Obtained avg loc unc of " + np.rad2deg(s.mean(locunc)) + "")
            if len(locunc)>0:
                self.localizationerrors.append(np.rad2deg(s.mean(locunc)))
            else:
                self.localizationerrors.append(1000)
                #some big # signifying that it can't catch it 

            print("avg loc unc  " +  str(s.mean(np.rad2deg(locunc))))

        return self.localizationerrors


The following cell contains the "FastCube" class. This is the simulation I hope to use to be able to run quicker simulations. 

In [9]:
import numpy as np
import healpy as hp
import burstutils as bf
import random as rand
import statistics as s

class FastCube():

    def __init__(self,background,dettilt,alternating=False):
        if alternating == False:
            self.tilt = np.deg2rad(dettilt)
            self.tiltA = self.tiltB = self.tiltC = self.tiltD = self.tilt
        
        else:
            self.tiltB = np.deg2rad(float(input("Please enter the second tilt (deg) ")))
        
            self.tiltC = self.tiltA = dettilt
            self.tiltD = self.tiltB
        self.zenith = [0 , 0]
        self.bg = background

            
    
    #make the normal vectors!
        #self.detA = [ zenith[0] + self.tiltA , zenith[1] ] 
        #self.detB = [ zenith[0] + self.tiltB , zenith[1] + np.pi/2 ] 
       # self.detC = [ zenith[0] + self.tiltC , zenith[1] + np.pi ] 
        #self.detD = [ zenith[0] + self.tiltD , zenith[1] + 3*np.pi/2 ] 

        #self.Anorm = hp.ang2vec(self.detA[0],self.detA[1])
        #self.Bnorm = hp.ang2vec(self.detB[0],self.detB[1])
        #self.Cnorm = hp.ang2vec(self.detC[0],self.detC[1])
        #self.Dnorm = hp.ang2vec(self.detD[0],self.detD[1])

    
       # self.dets = [self.Anorm,self.Bnorm,self.Cnorm,self.Dnorm] 
        
    @property
    def detA(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltA , self.zenith[1] ]
    @property 
    def detB(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltB , self.zenith[1] + np.pi/2 ]
    @property
    def detC(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltC , self.zenith[1] + np.pi ]
    @property 
    def detD(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltD , self.zenith[1] + 3*np.pi/2 ]
    @property
    def normA(self):
        return  hp.ang2vec(self.detA[0],self.detA[1])
    @property 
    def normB(self):
        return  hp.ang2vec(self.detB[0],self.detB[1])
    @property
    def normC(self):
        return  hp.ang2vec(self.detC[0],self.detC[1])
    @property 
    def normD(self):
        return  hp.ang2vec(self.detD[0],self.detD[1])

    
    @property
    def dets(self):
        return [self.normA,self.normB,self.normC,self.normD] 
    
    
    
    def response2GRB(self, GRB, samples = 1, test = False):   #is this how I inherit? 
        #first need to include the GRB.
       
        """
        Using least squares regression, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
        
        An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 

        Returns
        ----------
        localizationerrors : array

        numpy array that contains the average localization uncertainty at each sky position. 

        
        """
        
        if test:
            sample = 1
        else:
            sample = len(GRB.sourceangs)
        
        self.localizationerrors = []    
        for i in range(sample):
            sourceAng = GRB.sourceangs[i]
            print("Testing " + str(np.rad2deg(sourceAng)))
           #this check passes.       

            
           # print("Testing at " + str(np.rad2deg(GRB.sourceangs)))
            sourcexyz = hp.ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst
            loop = 0 #I'm going to want to sample each sky position more than once,
                    #here's where I define how many times that is
            locunc = []
            while loop<samples:
                sepA=bf.angle(sourcexyz,self.normA)
                   # print("separation is " + str(np.rad2deg(sep)))
                   #this check passes.  
               
                if sepA < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryA=bf.response(GRB.Ao,bf.angle(sourcexyz,self.normA),self.bg)  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryA = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                unccountsA = np.sqrt(dtheoryA)
                detactualA = rand.gauss(dtheoryA,unccountsA)  #there is a lot of noise, present, updating it now. 
                if detactualA-self.bg < 0:
                    detactualA = self.bg
                    
                detcountsA = dtheoryA  # I removed gaussian noise for now, have to readd after test. 
                
                sepB=bf.angle(sourcexyz,self.normB)
                   # print("separation is " + str(np.rad2deg(sep)))
                   #this check passes.  
               
                if sepB < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryB=bf.response(GRB.Ao,bf.angle(sourcexyz,self.normB),self.bg)  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryB = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                unccountsB = np.sqrt(dtheoryB) #another artifact, incl this background effect somewhere
                detactualB = rand.gauss(dtheoryB,unccountsB)  #there is a lot of noise, present, updating it now. 
                if detactualB-self.bg < 0:
                    detactualB = self.bg
                    
                detcountsB = dtheoryB

                sepC=bf.angle(sourcexyz,self.normC)
                   # print("separation is " + str(np.rad2deg(sep)))
                   #this check passes.  
               
                if sepC < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryC=bf.response(GRB.Ao,bf.angle(sourcexyz,self.normC),self.bg)  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryC = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                unccountsC = np.sqrt(dtheoryC)
                detactualC = rand.gauss(dtheoryC,unccountsC)  #there is a lot of noise, present, updating it now. 
                if detactualC-self.bg < 0:
                    detactualC = self.bg
                    
                detcountsC = dtheoryC
                
                sepD=bf.angle(sourcexyz,self.normD)
                   # print("separation is " + str(np.rad2deg(sep)))
                   #this check passes.  
               
                if sepD < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryD=bf.response(GRB.Ao,bf.angle(sourcexyz,self.normD),self.bg)  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryD = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                unccountsD = np.sqrt(dtheoryD)
                detactualD = rand.gauss(dtheoryD,unccountsD)  #there is a lot of noise, present, updating it now. 
                if detactualD-self.bg < 0:
                    detactualD = self.bg
                    
                detcountsD = dtheoryD
                
                
                chiA = bf.quad_solver(detcountsA,self.normA,0,90,0,360,self.bg)
                chiB = bf.quad_solver(detcountsB,self.normB,0,90,0,360,self.bg)
                chiC = bf.quad_solver(detcountsC,self.normC,0,90,0,360,self.bg)
                chiD = bf.quad_solver(detcountsD,self.normD,0,90,0,360,self.bg)
                
                chisquared = np.add(np.add(chiA,chiB),np.add(chiC,chiD)) #adds it all up for total chi2
                
                print(chisquared)
                
                #now obtain thetaloc
                chimin=np.amin(chisquared)
                chisquareds=list(chisquared)
                print("Index of chimin " + str(chisquareds.index(chimin)))
                
                """Aftr getting the minimum value in the chisquareds array, the commented out functions below are supposed to find where in the theta,phi, and A arrays what this corresponds to. It doesn't. 
                """
                oa = np.linspace(0,90,3)
                ob = np.linspace(0,360,4)
                Aofit = np.linspace(0,1000,5)
                thetaloc = np.rad2deg(oa[int((chisquareds.index(chimin)-(chisquareds.index(chimin) % (len(ob)*len(Aofit))))/(len(ob)*len(Aofit)))])
                philoc = np.rad2deg(ob[int(((chisquareds.index(chimin) % (len(ob)*len(Aofit)))-(chisquareds.index(chimin) % (len(Aofit))))/len(Aofit))])
                Aoguess=Aofit[int((chisquareds.index(chimin) % (len(ob)*len(Aofit)))  % len(Aofit))]
                if thetaloc > 180:
                    print("it recovered an unrealistic answer, skip")
                    break
                elif thetaloc < 0:
                    print("Same issue, there are limits to theta that are broken here. ")
                    break 
               # recpos = [thetaloc,philoc]
              #  print("Recovered position at " + str(recpos))
                recvec = hp.ang2vec(np.deg2rad(thetaloc),np.deg2rad(philoc))
                locunc.append(bf.angle(sourcexyz,recvec))
                loop +=1
            print("avg loc unc  " +  str(s.mean(np.rad2deg(locunc))))

        #return thetaloc

In [10]:
GRBtester = Sky(8,500)  #for both the fast and slow simulation, going to use an NSIDE=8 and strength =500 

The most important part of these classes is the "response2GRB" function. As you'll see in both, they actually happen to print out different results. There's a lot more they can do too, but for now I feel it best demonstrates what they're supposed to do by having the working one print out the localization uncertainty at each sky position, and the broken one print out where it thinks in the big array of possible values what the localized position is. For "FastCube" This is typically 2, which would correspond to something at the zenith, which would be incorrect.


To see what I'm talking about, simply run the cells below:

-They both correspond to BurstCube models where all detectors are evenly spaced and tilted down by 45°, with a background of 1000 counts. 

In [11]:
cube1 = SlowCube(1000,45)  #the first burstcube sim is here
cube1.response2GRB(GRBtester, test=True)  #just run this cell, you'll see that as different positions in the sky are tested,
# you'll see that the average localizations of a source is often <6°. 

Testing [  5.85026706  45.        ]
Det val: 1403.59171329
Det val: 1403.59171329
Det val: 1361.43537143
Det val: 1361.43537143
[424.00774621020298, 433.02141731488462, 424.00774621020298, 433.02141731488462, 4242.5120662369472, 4221.2995180734333, 4242.5120662369472, 4221.2995180734333]
avg loc unc  5.8502670637


[5.8502670637033551]

In [12]:
cube2 = FastCube(1000,45)  #same things

In [13]:
cube2.response2GRB(GRBtester, test=True)

Testing [  5.85026706  45.        ]
Detector val: 1403.59171329
separation here, is it okay? [ 45.  45.  45.  45.]
Detector val: 1403.59171329
separation here, is it okay? [ 45.  45.  90.  90.]
Detector val: 1361.43537143
separation here, is it okay? [  45.   45.  135.  135.]


ValueError: operands could not be broadcast together with shapes (8,) (4,) 

At this point I'm more concerned with what is being printed out 

In [8]:
stuff = bf.solver(cube1.detcounts,cube1.dets,0,90,0,360,10,37,cube1.bg)

Det val: 1403.59171329
Det val: 1403.59171329
Det val: 1361.43537143
Det val: 1361.43537143


In [ ]:
np.min(stuff[3])

In [ ]:
np.min?

In [ ]:
thetas = np.linspace(0,90,2)
phis = np.linspace(0,360,2)

In [ ]:
xx,yy = np.meshgrid(phis,thetas)

In [ ]:
np.concatenate(xx)

In [ ]:
np.concatenate(yy)

In [ ]:
hp.vec2ang(hp.ang2vec(np.deg2rad(np.concatenate(yy)),np.deg2rad(np.concatenate(xx))))   #check holds in this meshgrid. 

In [ ]:
theta = np.deg2rad(np.linspace(0,45,2))
phi = np.deg2rad(np.linspace(0,90,2))
mphi,mtheta = np.meshgrid(phi,theta)
allthetas = np.concatenate(mtheta)
allphis = np.concatenate(mphi)
allvecs = hp.ang2vec(allthetas,allphis)
As= np.linspace(0,1000,2)
normarr = [0,1/np.sqrt(2),1/np.sqrt(2)]
normarrs = []
for garc in range((len(theta)*len(phi))):
    normarrs.append([normarr[0],normarr[1],normarr[2]])
        
seps = bf.findAngles(allvecs,normarrs)

In [ ]:
np.rad2deg(seps)

In [ ]:
np.linspace(0,180,2)

In [ ]:
import astropy

In [ ]:
x+x

In [ ]:
x

In [ ]:
np.divide(x,x)

In [ ]:
np.power(x,3)

In [ ]:
np.divide(np.power(np.subtract(chiResponse,detval),2),detval)

In [ ]:
theta = np.deg2rad(np.linspace(0,90,2))
phi = np.deg2rad(np.linspace(0,180,2))
mphi,mtheta = np.meshgrid(phi,theta)
allthetas = np.concatenate(mtheta)
allphis = np.concatenate(mphi)
allvecs = hp.ang2vec(allthetas,allphis)

In [ ]:
allvecs

In [ ]:
normarr = cube2.normA
normarrs = []
for garc in range((len(theta)*len(phi))):
    normarrs.append([normarr[0],normarr[1],normarr[2]])

In [ ]:
normarrs

In [ ]:
np.rad2deg(bf.findAngles(allvecs,normarrs))

In [ ]:
def chiresponse(A):
    """
    Meant to imitate the actual response of a scintillator.
    Inputs 2 vectors, and responds with a cos^x dependence.
    """
    #meant to imitate the response of the detectors for effective area vs. angle, found to be around .77
 #   print(length(A),length(B))
#if cosine is negative, 

    mask = A > np.pi/2.

    A[mask] = 0
    A[~mask] = pow(abs(np.cos(A[~mask])),0.76)
    
    
    return A

In [ ]:
chiresponse(np.array([0,np.pi,np.pi/4,np.pi/2,np.pi/6]))